# The General Hurricane claim project Team cat_5

## import packages for Geospatial and webscraping

In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

In [3]:
# read in .csv file to review
cat_loss_data = pd.read_csv('..\data\Cat Loss Data.csv')
cat_loss_data.columns = cat_loss_data.columns.str.strip().str.lower()
cat_loss_data.columns = cat_loss_data.columns.str.replace(' ', '_')

In [4]:
cat_loss_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   policy_number       1020 non-null   object 
 1   claim_number        1020 non-null   object 
 2   garaging_city       1020 non-null   object 
 3   garaging_zip        1020 non-null   int64  
 4   garaging_state      1020 non-null   object 
 5   loss_date           1020 non-null   object 
 6   loss_date_time      1020 non-null   object 
 7   fnol_date           1020 non-null   object 
 8   fnol_time           1020 non-null   object 
 9   cat_name            1020 non-null   object 
 10  cat_number          1020 non-null   object 
 11  loss_description    1020 non-null   object 
 12  loss_cause          1020 non-null   object 
 13  loss_location_city  1020 non-null   object 
 14  loss_location_zip   834 non-null    object 
 15  loss_state          1020 non-null   object 
 16  year  

In [5]:
cat_loss_data.head()

,policy_number,claim_number,garaging_city,garaging_zip,garaging_state,loss_date,loss_date_time,fnol_date,fnol_time,cat_name,...,acv,coverage,exp_team,losses_paid,est_received_amt,deductible,gross_estimate_amt,payment_ind,salvage,tl_settlement_amt
0,POL00541330,CLAIM0014123,Apopka,32703,FL,2016-10-07,10:30:00,2016-10-07,12:46:06,Hurrican Matthew,...,11297,COMP,Team Kiggins,7315.75,8979.30,500,9479.30,CWP,0.0,0.0
1,POL00541632,CLAIM0014127,DAYTONA BEACH,32114,FL,2016-10-07,13:00:00,2016-10-07,14:35:55,Hurrican Matthew,...,0,COMP,Team Suiter,0.00,0.00,1000,1000.00,CWOP,0.0,0.0
2,POL00541734,CLAIM0014131,ALTAMONTE SPRINGS,32714,FL,2016-10-07,00:01:00,2016-10-07,16:17:08,Hurrican Matthew,...,0,COMP,Team Bowden TAG,288.57,0.00,1000,0.00,CWP,0.0,0.0
3,POL00543202,CLAIM0014132,PORT SAINT LUCIE,34952,FL,2016-10-07,00:00:00,2016-10-07,16:33:08,Hurrican Matthew,...,0,COMP,Team Bowden TAG,0.00,0.00,500,0.00,CWOP,0.0,0.0
4,POL00545825,CLAIM0014135,DEBARY,32713,FL,2016-10-07,00:01:00,2016-10-07,17:34:20,Hurrican Matthew,...,0,COMP,Team Suiter,576.58,576.58,500,1076.58,CWP,0.0,0.0


In [6]:
cat_loss_data.count()

policy_number         1020
claim_number          1020
garaging_city         1020
garaging_zip          1020
garaging_state        1020
loss_date             1020
loss_date_time        1020
fnol_date             1020
fnol_time             1020
cat_name              1020
cat_number            1020
loss_description      1020
loss_cause            1020
loss_location_city    1020
loss_location_zip      834
loss_state            1020
year                  1020
make                  1020
model                 1019
total_loss            1020
mileage               1020
vehicle_origin         363
acv                   1020
coverage               781
exp_team              1020
losses_paid           1020
est_received_amt      1020
deductible            1020
gross_estimate_amt    1020
payment_ind            781
salvage               1020
tl_settlement_amt     1020
dtype: int64

In [7]:
loss_city = cat_loss_data.groupby('loss_location_city')
loss_city.head()

,policy_number,claim_number,garaging_city,garaging_zip,garaging_state,loss_date,loss_date_time,fnol_date,fnol_time,cat_name,...,acv,coverage,exp_team,losses_paid,est_received_amt,deductible,gross_estimate_amt,payment_ind,salvage,tl_settlement_amt
0,POL00541330,CLAIM0014123,Apopka,32703,FL,2016-10-07,10:30:00,2016-10-07,12:46:06,Hurrican Matthew,...,11297,COMP,Team Kiggins,7315.75,8979.30,500,9479.30,CWP,0.0,0.0
1,POL00541632,CLAIM0014127,DAYTONA BEACH,32114,FL,2016-10-07,13:00:00,2016-10-07,14:35:55,Hurrican Matthew,...,0,COMP,Team Suiter,0.00,0.00,1000,1000.00,CWOP,0.0,0.0
2,POL00541734,CLAIM0014131,ALTAMONTE SPRINGS,32714,FL,2016-10-07,00:01:00,2016-10-07,16:17:08,Hurrican Matthew,...,0,COMP,Team Bowden TAG,288.57,0.00,1000,0.00,CWP,0.0,0.0
3,POL00543202,CLAIM0014132,PORT SAINT LUCIE,34952,FL,2016-10-07,00:00:00,2016-10-07,16:33:08,Hurrican Matthew,...,0,COMP,Team Bowden TAG,0.00,0.00,500,0.00,CWOP,0.0,0.0
4,POL00545825,CLAIM0014135,DEBARY,32713,FL,2016-10-07,00:01:00,2016-10-07,17:34:20,Hurrican Matthew,...,0,COMP,Team Suiter,576.58,576.58,500,1076.58,CWP,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,POL01740491,CLAIM0054366,NORTH CHARLESTON,29405,SC,2019-09-04,06:00:00,2019-09-09,14:36:03,Hurricane Dorian,...,0,NaN,Team Kimble,0.00,0.00,0,0.00,NaN,0.0,0.0
1012,POL00629390,CLAIM0054422,EDGEWATER,32141,FL,2019-09-02,22:00:00,2019-09-10,20:14:14,Hurricane Dorian,...,0,COMP,Team Bowden TAG,0.00,0.00,1000,0.00,CWOP,0.0,0.0
1014,POL01756281,CLAIM0054492,MYRTLE BEACH,29588,SC,2019-09-05,00:01:00,2019-09-12,15:59:21,Hurricane Dorian,...,0,NaN,Team Bowden TAG,0.00,0.00,0,0.00,NaN,0.0,0.0
1017,POL01144223,CLAIM0054554,LELAND,28451,NC,2019-09-04,22:00:00,2019-09-14,13:45:30,Hurricane Dorian,...,0,COMP,Team Brown,0.00,731.59,500,1231.59,CWOP,0.0,0.0


In [8]:
print(cat_loss_data['policy_number'].unique())


['POL00541330' 'POL00541632' 'POL00541734' 'POL00543202' 'POL00545825'
 'POL00542757' 'POL00559271' 'POL01738501' 'POL02033909' 'POL01738717'
 'POL01745073' 'POL00552392' 'POL00548540' 'POL01739197' 'POL01745319'
 'POL01577574' 'POL00549385' 'POL00558380' 'POL02039520' 'POL00553103'
 'POL01744856' 'POL01745524' 'POL02043916' 'POL01741238' 'POL01744700'
 'POL01764398' 'POL02105515' 'POL02042897' 'POL01109760' 'POL01109599'
 'POL01109562' 'POL02031471' 'POL02039490' 'POL02029631' 'POL01103335'
 'POL02039563' 'POL01740392' 'POL00554858' 'POL02045574' 'POL01739947'
 'POL01740866' 'POL01738832' 'POL00688278' 'POL01738734' 'POL00540725'
 'POL00548627' 'POL01744781' 'POL02042534' 'POL02041917' 'POL00545338'
 'POL01738979' 'POL01738611' 'POL01764609' 'POL02040458' 'POL00542434'
 'POL01739569' 'POL00554645' 'POL00556842' 'POL01744167' 'POL00558025'
 'POL02105692' 'POL02041545' 'POL01737952' 'POL00541466' 'POL01743814'
 'POL01738705' 'POL00558185' 'POL01745399' 'POL02040963' 'POL02028574'
 'POL0

In [9]:
cat_loss_data.cat_name.unique()

array(['Hurrican Matthew', 'Hurricane Harvey', 'Hurricane Irma',
       'Hurricane Nate', 'Hurricane Florence', 'Hurricane Michael',
       'Hurricane Dorian'], dtype=object)

In [ ]:
# next steps -  clean Hurrican Matthew

Data['loss_location_zip'] = Data['loss_location_zip'].str[:5]